In [2]:
import pickle
import numpy as np
import pandas as pd
from functools import reduce
import os

Old try #1
```python
print(len(retweet_network))
print(retweet_network[4])
print(np.shape([len(dct["retweeted_users_ids"]) > 2 for dct in retweet_network]))
print(sum([len(dct["retweeted_users_ids"]) > 4 for dct in retweet_network]))

with open(r"F:\TwitterUsers\archiveTeam\processed_files\retweet_network.txt","rb") as f:
    retweet_network = pickle.load(f)
    f.close()
    
    
with open(r"F:\TwitterUsers\archiveTeam\saved_objs\10\objs_0.txt","rb") as f:
    day = pickle.load(f)
    f.close()
    
print(day[0])
with open(r"F:\TwitterUsers\archiveTeam\processed_files\unsubscribed_users.txt","rb") as f:
    unsubscribed_users = set(pickle.load(f))
    f.close()

print(unsubscribed_users[1])
with open(r"F:\TwitterUsers\archiveTeam\processed_files\ok_users.txt","rb") as f:
    ok_users = set(pickle.load(f))
    f.close()

print(ok_users[2])
```

Old try #2
```python
%%time
with open(r"F:\TwitterUsers\archiveTeam\saved_objs\01\objs_0.txt","rb") as f:
    day0_lst = pickle.load(f)
    f.close()

with open(r"F:\TwitterUsers\archiveTeam\processed_files\hashtags_3.txt", "rb") as f:
    hashtags = set(pickle.load(f))
    f.close()
print(len(hashtags))
day0_csv = pd.DataFrame(day0_lst).drop(labels = ["tweet_created_at","followers_count","friends_count","statuses_count"], axis  = 1)
print(len(day0_csv))
day0_csv["retweeted_tweet_id"] = day0_csv["retweeted_tweet_id"].apply(encapsulate)
day0_csv["retweeted_user_id"] = day0_csv["retweeted_user_id"].apply(encapsulate)
day0_csv= day0_csv[ get_lists_indices(day0_csv["tweet_id"])]
day0_csv.reset_index(inplace = True, drop = True)
day0_csv = day0_csv[select_by_hashtags(day0_csv["hashtags"])]
print(len(day0_csv))
day0_csv_agg = day0_csv.groupby(["user_id","profile_created_at"], as_index = False).agg({"retweeted_tweet_id":"sum","retweeted_user_id":"sum","hashtags":"sum"})
day0_csv_agg.head(50)
#print(len(day0_csv_agg))
```

Define useful functions

In [6]:
def get_lists_indices(series):
    return [type(elem) != list for elem in series  ]

def reduce_extend(series):
    return reduce(lambda x,y: x.extend(y), series)

def encapsulate(x):
    if type(x) == list:
        return x
    else:
        return [x]
    
def isNone(series):
    return [x is None for x in series]

def select_by_hashtags(series):
    return [not set([hashtag.lower() for hashtag in lst]).isdisjoint(hashtags) for lst in series]

Load paths and hashtags

In [19]:
listOfFiles = list()
for (dirpath, dirnames, filenames) in os.walk(r"G:\TwitterUsers\saved_objs_withKeywords"):
    listOfFiles += [os.path.join(dirpath, file) for file in filenames]
total = len(listOfFiles)
print(total)

with open(r"G:\TwitterUsers\processed_files\hashtags\hashtags_1_filtered.txt", "rb") as f:
    hashtags = set(pickle.load(f))
    f.close()
print(len(hashtags))

330
6948


In [22]:
# with open(r"G:\TwitterUsers\saved_objs_withKeywords\11\objs_0.txt","rb") as f:
#     day_csv = pd.DataFrame(pickle.load(f)).drop(labels = ["tweet_created_at","followers_count","friends_count","statuses_count","favourites_count","text"], axis  = 1)
#     f.close()
# day_csv.head()
# df = pd.DataFrame()
# day_csv["retweeted_tweet_id"] = day_csv["retweeted_tweet_id"].apply(encapsulate)
# day_csv["retweeted_user_id"] = day_csv["retweeted_user_id"].apply(encapsulate)
# day_csv= day_csv[ get_lists_indices(day_csv["tweet_id"])]
# day_csv = day_csv[select_by_hashtags(day_csv["hashtags"])]
# df = df.append(day_csv)
# df = df.groupby(["user_id","profile_created_at"] ,as_index = False).agg({"retweeted_tweet_id":"sum","retweeted_user_id":"sum","hashtags":"sum"})
# df.reset_index(inplace = True, drop = True)
# df.head()

In [9]:
# day_csv["retweeted_tweet_id"] = day_csv["retweeted_tweet_id"].apply(encapsulate)
# day_csv["retweeted_user_id"] = day_csv["retweeted_user_id"].apply(encapsulate)
# day_csv= day_csv[ get_lists_indices(day_csv["tweet_id"])]
# day_csv = day_csv[select_by_hashtags(day_csv["hashtags"])]
# my_df = pd.DataFrame({"a":[[],[12]],"b":[[1,2,3],[1,2,3,4]],"c":[[1],[8,9]]})
# my_df
# my_df["d"] = my_df["a"]+my_df["b"]+my_df["c"]
# my_df
# day_csv.head(20)

Process and save data

In [ ]:
%%time
df = pd.DataFrame()
i = 0
save_path = r"G:\TwitterUsers\processed_files\retweet_network\retweet_network.npy"
backup_path = r"G:\TwitterUsers\processed_files\retweet_network\backup\retweet_network.npy"
for path in  listOfFiles:
    with open(path,"rb") as f:
        
        day_csv = pd.DataFrame(pickle.load(f)).drop(labels = ["tweet_created_at","followers_count","friends_count","statuses_count","favourites_count","text"], axis  = 1)
        f.close()
    #day0_csv = pd.DataFrame(day0_lst).drop(labels = ["tweet_created_at","followers_count","friends_count","statuses_count"], axis  = 1)
    #print(len(day0_csv))
    day_csv["retweeted_tweet_id"] = day_csv["retweeted_tweet_id"].apply(encapsulate)
    day_csv["retweeted_user_id"]  = day_csv["retweeted_user_id"].apply(encapsulate)
    day_csv = day_csv[ get_lists_indices(day_csv["tweet_id"])]
    #day_csv.reset_index(inplace = True, drop = True)
    day_csv = day_csv[select_by_hashtags(day_csv["hashtags"])]
#     try:
    df = df.append(day_csv)
#     except:
#         df.drop(labels = ["user_mentions","replied_to_tweet_id","replied_to_user_id"])
#         df = df.append(day_csv)
    df = df.groupby(["user_id","profile_created_at"] ,as_index = False).agg({"retweeted_tweet_id":"sum","retweeted_user_id":"sum","hashtags":"sum"}) #,"user_mentions":"sum" (teh columns not specified in agg are removed)
    #df["interacted_users_ids"] = day_csv["retweeted_user_id"] + day_csv["user_mentions"]
    df.reset_index(inplace = True, drop = True)
    i += 1
    print("done", i,"day(s) out of", total, ", len(df) =",len(df), end = "\r")
    if i%10 == 0:
        print("saving...", end = "\r")
        df_np = df.to_numpy()
        np.save(save_path, df_np)
        np.save(backup_path, df_np)
        
print("last saving...")
df_np = df.to_numpy()
np.save(save_path, df_np)
np.save(backup_path, df_np)
    #print(len(day0_csv))

In [ ]:
# useless, all data was already saved. Do NOT uncomment
# save_path = r"F:\TwitterUsers\archiveTeam\retweet_network\retweet_network1.npy"
# backup_path = r"F:\TwitterUsers\archiveTeam\retweet_network\backup\retweet_network1.npy"
# df_np = df.to_numpy()
# np.save(save_path, df_np)
# np.save(backup_path, df_np)

Reload the numpy array and convert it to csv

In [ ]:
# retweet_network_np = np.load(r"F:\TwitterUsers\archiveTeam\retweet_network\retweet_network.npy", allow_pickle = True)
# retweet_network_csv = pd.DataFrame(retweet_network_np,columns = ["user_id",	"profile_created_at",	"retweeted_tweet_id",	"retweeted_user_id",	"hashtags"] )
# print(retweet_network_np.shape)
# retweet_network_csv.head(50)